In [1]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [2]:
# Imports

import nflreadpy as nfl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Retrieve data

In [3]:
schedule = nfl.load_schedules(seasons=[2024, 2025])
schedule = schedule.to_pandas()
schedule = schedule[schedule['game_type'] == 'REG']

team_stats = nfl.load_team_stats(seasons=[2024, 2025])
team_stats = team_stats.to_pandas()
team_stats = team_stats[team_stats['season_type'] == 'REG']

In [4]:
team_stats

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,2024,1,ARI,REG,BUF,21,31,162,1,0,...,2,2,0,0,1.000000,0,0,0,0,0
1,2024,1,ATL,REG,PIT,16,26,155,1,2,...,1,1,0,0,1.000000,0,0,0,0,0
2,2024,1,BAL,REG,KC,26,41,273,1,0,...,2,2,0,0,1.000000,0,0,0,0,0
3,2024,1,BUF,REG,ARI,18,23,232,2,0,...,4,4,0,0,1.000000,0,0,0,0,0
4,2024,1,CAR,REG,NO,13,31,161,0,2,...,1,1,0,0,1.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,2025,10,PIT,REG,LAC,16,31,161,1,2,...,1,1,0,0,1.000000,0,0,0,0,0
862,2025,10,SEA,REG,ARI,10,12,178,1,1,...,5,5,0,0,1.000000,0,0,0,0,0
863,2025,10,SF,REG,LA,33,39,319,3,1,...,2,3,0,1,0.666667,0,0,0,0,0
864,2025,10,TB,REG,NE,28,43,273,3,0,...,2,2,0,0,1.000000,0,0,0,0,0


In [5]:
# Filter out data I don't want

cols_to_drop = [
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [6]:
# for each week group by team sum/avg all previous weeks , separate into 2024 and 2025 seasons
team_stats_24 = team_stats[team_stats['season'] == 2024]
team_stats_25 = team_stats[team_stats['season'] == 2025]
team_stats_24 = team_stats_24.sort_values(["team", "week"]).reset_index(drop=True)
team_stats_25 = team_stats_25.sort_values(["team", "week"]).reset_index(drop=True)

In [7]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att']


In [8]:
team_stats_24['week'] = team_stats_24['week'].astype(int)

# Get all unique teams and full week range
teams = team_stats_24['team'].unique()
weeks = range(1, team_stats_24['week'].max() + 1)

# Create a MultiIndex of all possible (team, week) combinations
full_index = pd.MultiIndex.from_product([teams, weeks], names=['team', 'week'])

# Reindex the dataframe to include all rows
team_stats_24 = team_stats_24.set_index(['team', 'week']).reindex(full_index).reset_index()

In [9]:
# Compute cumulative mean for previous games (excluding current game)
team_stats_24[[f"avg_{c}" for c in avg_cols]] = (
    team_stats_24.sort_values(by=['team', 'week'])
    .groupby("team")[avg_cols]
      .expanding()
      .mean()
      .shift(periods=1)
      .reset_index(level=0, drop=True)
)

In [10]:
team_stats_25['week'] = team_stats_25['week'].astype(int)

# Get all unique teams and full week range
teams = team_stats_25['team'].unique()
weeks = range(1, team_stats_25['week'].max() + 1)

# Create a MultiIndex of all possible (team, week) combinations
full_index = pd.MultiIndex.from_product([teams, weeks], names=['team', 'week'])

# Reindex the dataframe to include all rows
team_stats_25 = team_stats_25.set_index(['team', 'week']).reindex(full_index).reset_index()

In [11]:
# Compute cumulative mean for previous games (excluding current game)
team_stats_25[[f"avg_{c}" for c in avg_cols]] = (
    team_stats_25.sort_values(by=['team', 'week'])
    .groupby("team")[avg_cols]
      .expanding()
      .mean()
      .shift(periods=1)
      .reset_index(level=0, drop=True)
)

## Retrieve Time of Possesion Stats

In [12]:
for attempt in range(1, 5 + 1):
    try:
        # set up the browser
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        
        # visit T.O.P. Website
        driver.get("https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot")

        table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='DataTables_Table_0']"))
                    )

        df = pd.read_html(table.get_attribute("outerHTML"))[0]

        driver.quit()
        break

    except Exception as e:
        driver.quit()
        print(f"Attempt {attempt} failed: {e}")
        if attempt == 5:
            print("No retries left — raising.")
            raise

C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\26274015.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


In [13]:
mapping = {'Buffalo': 'BUF',
 'LA Chargers': 'LAC',
 'Kansas City': 'KC',
 'Houston': 'HOU',
 'Detroit': 'DET',
 'Carolina': 'CAR',
 'Arizona': 'ARI',
 'New England': 'NE',
 'Jacksonville': 'JAX',
 'Chicago': 'CHI',
 'Tampa Bay': 'TB',
 'Atlanta': 'ATL',
 'NY Giants': 'NYG',
 'Denver': 'DEN',
 'Green Bay': 'GB',
 'San Francisco': 'SF',
 'Philadelphia': 'PHI',
 'Cleveland': 'CLE',
 'Dallas': 'DAL',
 'NY Jets': 'NYJ',
 'Indianapolis': 'IND',
 'Las Vegas': 'LV',
 'Minnesota': 'MIN',
 'Seattle': 'SEA',
 'Miami': 'MIA',
 'LA Rams': 'LA',
 'Baltimore': 'BAL',
 'New Orleans': 'NO',
 'Tennessee': 'TEN',
 'Washington': 'WAS',
 'Pittsburgh': 'PIT',
 'Cincinnati': 'CIN'}

df['team_abb'] = df['Team'].map(mapping)

In [14]:
top_dict_24 = pd.Series(df['2024'].values, index=df['team_abb']).to_dict()
top_dict_25 = pd.Series(df['2025'].values, index=df['team_abb']).to_dict()

## Add stats for previous weeks in schedule

In [15]:
desired_columns = ['season', 'week', 'team', 'opponent_team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist',
       'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits',
       'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

team_stats_24 = team_stats_24[desired_columns]
team_stats_25 = team_stats_25[desired_columns]


In [16]:
team_stats = pd.concat([team_stats_24, team_stats_25])

In [17]:
offensive = ['season','week', 'team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

defensive = ['season', 'week', 'opponent_team', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist', 'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 
        'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits', 'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 
        'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties']

offensive_stats = team_stats[offensive]
defensive_stats = team_stats[defensive]

In [18]:
# columns from schedule: week, home_team, away_team, home_score, away_score, total_line
schedule = schedule[schedule['week'] > 1]
schedule_interests = schedule[['game_id', 'season', 'week', 'home_team', 'home_score', 'away_team', 'away_score', 'total_line']]

In [19]:
schedule_away = schedule_interests
schedule_home = schedule_interests

In [20]:
schedule_away = schedule_away.rename(columns={'home_team':'opponent', 'away_team':'team', 'away_score':'points'})
schedule_away = schedule_away[['game_id', 'season', 'week', 'team', 'opponent', 'points', 'total_line']]

schedule_home = schedule_home.rename(columns={'away_team':'opponent', 'home_team':'team', 'home_score':'points'})
schedule_home = schedule_home[['game_id', 'season', 'week', 'team', 'opponent', 'points', 'total_line']]

In [21]:
schedule_total = pd.concat([schedule_away, schedule_home])

In [22]:
schedule_total

,game_id,season,week,team,opponent,points,total_line
16,2024_02_BUF_MIA,2024,2,BUF,MIA,31.0,48.5
17,2024_02_LV_BAL,2024,2,LV,BAL,26.0,41.0
18,2024_02_LAC_CAR,2024,2,LAC,CAR,26.0,39.0
19,2024_02_NO_DAL,2024,2,NO,DAL,44.0,47.0
20,2024_02_TB_DET,2024,2,TB,DET,20.0,51.5
...,...,...,...,...,...,...,...
552,2025_18_DAL_NYG,2025,18,NYG,DAL,NaN,NaN
553,2025_18_WAS_PHI,2025,18,PHI,WAS,NaN,NaN
554,2025_18_BAL_PIT,2025,18,PIT,BAL,NaN,NaN
555,2025_18_SEA_SF,2025,18,SF,SEA,NaN,NaN


In [23]:
offensive_stats

,season,week,team,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,avg_receiving_epa,avg_fg_pct,avg_pat_pct,avg_completions,...,avg_rushing_fumbles_lost,avg_rushing_first_downs,avg_receiving_fumbles_lost,avg_penalties,avg_penalty_yards,avg_punt_returns,avg_punt_return_yards,avg_kickoff_returns,avg_kickoff_return_yards,avg_fg_att
0,2024.0,1,ARI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024.0,2,ARI,-1.184850,1.196628,4.984950,8.129718,1.00,1.0,21.000000,...,0.000000,7.000000,0.00,5.000000,31.000000,1.000000,6.000000,3.000000,123.000000,2.000000
2,2024.0,3,ARI,8.458119,9.371844,4.193653,14.001905,1.00,1.0,19.000000,...,0.500000,11.000000,0.00,5.000000,41.500000,1.500000,18.000000,1.500000,61.500000,2.000000
3,2024.0,4,ARI,3.761445,5.712925,0.461242,7.908119,1.00,1.0,19.666667,...,0.333333,8.000000,0.00,4.666667,37.666667,1.000000,12.000000,1.000000,41.000000,2.000000
4,2024.0,5,ARI,2.712770,4.387988,0.604399,6.289543,1.00,1.0,18.750000,...,0.250000,9.250000,0.25,4.500000,48.750000,1.000000,10.000000,2.000000,63.250000,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,2025.0,6,WAS,3.389712,1.695046,2.802895,8.699414,0.75,1.0,17.800000,...,0.400000,7.600000,0.00,7.400000,54.200000,2.400000,35.400000,3.600000,119.600000,2.400000
316,2025.0,7,WAS,3.897319,3.270358,0.636833,9.342742,0.70,1.0,18.000000,...,0.666667,7.666667,0.00,7.000000,51.833333,2.166667,31.000000,3.666667,120.833333,2.333333
317,2025.0,8,WAS,0.489855,1.102632,1.373142,6.596160,0.70,1.0,17.714286,...,0.571429,7.714286,0.00,7.571429,61.285714,2.000000,27.714286,4.285714,133.428571,2.000000
318,2025.0,9,WAS,-0.496898,1.105433,0.950164,5.384997,0.70,1.0,18.125000,...,0.500000,7.250000,0.00,7.000000,58.000000,1.875000,26.000000,4.125000,125.125000,1.750000


In [24]:
schedule_total['team'] = schedule_total['team'].str.strip().str.upper()
offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
schedule_total['opponent'] = schedule_total['opponent'].str.strip().str.upper()
defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()

result = schedule_total.merge(offensive_stats, on=['season', 'week', 'team'], how='left')
result = result.merge(defensive_stats, left_on=['season', 'week', 'opponent'], right_on=['season', 'week','opponent_team'], how='left')

C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\4226889885.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_stats['team'] = offensive_stats['team'].str.strip().str.upper()
C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\4226889885.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_stats['opponent_team'] = defensive_stats['opponent_team'].str.strip().str.upper()


### Final Touches and Download Data as CSV

In [25]:
# Columns to fill
cols_to_fill = ['avg_fg_pct', 'avg_pat_pct']

# Fill NaN with 1 only for week 2 due to teams not having attempted a fg prior to the week
result.loc[result['week'] == 2, cols_to_fill] = result.loc[result['week'] == 2, cols_to_fill].fillna(1)


In [26]:
# Split result into '24 and '25
result_24 = result[result['season'] == 2024]
result_25 = result[result['season'] == 2025]

In [27]:
# Map time of possession stats
result_24['team_top'] = result_24['team'].map(top_dict_24)
result_24['opp_top'] = result_24['opponent_team'].map(top_dict_24)
result_25['team_top'] = result_25['team'].map(top_dict_25)
result_25['opp_top'] = result_25['opponent_team'].map(top_dict_25)

C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\3603531570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_24['team_top'] = result_24['team'].map(top_dict_24)
C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\3603531570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_24['opp_top'] = result_24['opponent_team'].map(top_dict_24)
C:\Users\tyler\AppData\Local\Temp\ipykernel_46716\3603531570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [29]:
# Recombine the results years
result = pd.concat([result_24, result_25])

In [30]:
result

,game_id,season,week,team,opponent,points,total_line,avg_passing_epa,avg_passing_cpoe,avg_rushing_epa,...,avg_def_sack_yards,avg_def_qb_hits,avg_def_interceptions,avg_def_interception_yards,avg_def_pass_defended,avg_def_tds,avg_def_fumbles,avg_def_safeties,team_top,opp_top
0,2024_02_BUF_MIA,2024,2,BUF,MIA,31.0,48.5,9.161098,9.891942,3.497710,...,16.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,30:34,31:36
1,2024_02_LV_BAL,2024,2,LV,BAL,26.0,41.0,-1.242705,1.044023,-18.512759,...,4.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,29:20,31:12
2,2024_02_LAC_CAR,2024,2,LAC,CAR,26.0,39.0,-4.875880,-0.624557,0.271463,...,22.0,6.0,1.0,3.0,5.0,0.0,0.0,0.0,30:23,27:13
3,2024_02_NO_DAL,2024,2,NO,DAL,44.0,47.0,7.993491,10.192335,4.767348,...,26.0,5.0,2.0,2.0,6.0,0.0,1.0,0.0,28:08,29:16
4,2024_02_TB_DET,2024,2,TB,DET,20.0,51.5,20.591765,12.676064,-6.825627,...,23.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,31:41,31:46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,2025_18_DAL_NYG,2025,18,NYG,DAL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:11,NaN
1020,2025_18_WAS_PHI,2025,18,PHI,WAS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:00,NaN
1021,2025_18_BAL_PIT,2025,18,PIT,BAL,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27:00,NaN
1022,2025_18_SEA_SF,2025,18,SF,SEA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30:09,NaN


In [31]:
import os

try:
    # Works when running as a .py script
    base_dir = os.path.dirname(__file__)
except NameError:
    # Fallback for Jupyter, Python in Excel, or interactive environments
    base_dir = os.getcwd()

data_path = os.path.join(base_dir, '..', 'Data', 'NFLData.csv')

os.makedirs(os.path.dirname(data_path), exist_ok=True)
result.to_csv(data_path, index=False)